In [97]:
import pandas as pd
import numpy as np
from faker import Faker
import hashlib

In [98]:
# %pip install faker

In [99]:
df = pd.read_excel('data/arquivo_anonimizado.xlsx')
df.drop(['Unnamed: 0', 'reprovacoes'], axis=1, inplace=True)

In [100]:
faker = Faker('pt_BR')

quantidade_de_nomes = df.shape[0]
nomes_completos = [faker.name() for _ in range(quantidade_de_nomes)]

df['nome_completo'] = nomes_completos

In [101]:
df.head(5).T

,0,1,2,3,4
matriculaDRE,0,1,2,3,4
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por abandono,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 062...,2000/1 - MAA123 Algebra para Informatica - 009...,2000/1 - MAA123 Algebra para Informatica - 040...,2000/1 - MAA123 Algebra para Informatica - 081...
crPorPeriodo,2000/1 - 7.6\n2000/2 - 7.1\n2001/1 - 5.9\n2001...,2000/1 - 7.6\n2000/2 - 6.5\n2001/1 - 5.1\n2001...,2000/1 - 3.3\n2000/2 - 4.7\n2001/1 - 0.7,2000/1 - 6.3\n2000/2 - 5.1\n2001/1 - 2.8\n2001...,2000/1 - 8.3\n2000/2 - 7\n2001/1 - 7.2\n2001/2...
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7\n2001/1 - 6.4\n2001/2...,2000/1 - 3.3\n2000/2 - 3.9\n2001/1 - 3.2,2000/1 - 6.3\n2000/2 - 5.7\n2001/1 - 4.7\n2001...,2000/1 - 8.4\n2000/2 - 7.7\n2001/1 - 7.6\n2001...


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   matriculaDRE               2817 non-null   int64  
 1   periodoIngressoUFRJ        2817 non-null   object 
 2   cursoIngressoUFRJ          2817 non-null   object 
 3   cursoAtual                 2817 non-null   object 
 4   periodoIngressoCursoAtual  2817 non-null   object 
 5   situacaoMatriculaAtual     2817 non-null   object 
 6   sexo                       2817 non-null   object 
 7   disciplinasCursadas        2650 non-null   object 
 8   crPorPeriodo               2644 non-null   object 
 9   craPorPeriodo              2642 non-null   object 
 10  periodosIntegralizados     2817 non-null   int64  
 11  cargaHorariaAcumulada      2817 non-null   int64  
 12  CRA                        2654 non-null   float64
 13  periodosTrancados          948 non-null    objec

# Processamento

Removendo alunos que não possui disciplinas cursas, pois se não há disciplinas, não há metricas a serem analisadas.

In [103]:
df.drop(['CRA', 'periodosCRMenor3', 'periodosTrancados', 'periodosCancelados'], axis=1, inplace=True)
df.dropna(subset=['disciplinasCursadas'], inplace=True)

## CR por Periodo

In [104]:
df['crPorPeriodo'] = df['crPorPeriodo'].astype(str)
df['crPorPeriodo'] = df['crPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('crPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.1)","(2001/1, 5.9)","(2001/2, 6.8)","(2002/1, 6.3)"
craPorPeriodo,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...,2000/1 - 7.6\n2000/2 - 7.4\n2001/1 - 6.9\n2001...


In [105]:
df_crPeriodo = pd.DataFrame(df_exploded['crPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cr'))
df_crPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_crPeriodo

,periodo,cr,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.1,0
0,2001/1,5.9,0
0,2001/2,6.8,0
0,2002/1,6.3,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


In [106]:
# df_fato = pd.concat([df_exploded, df_crPeriodo], axis=1)
# df_fato.drop('crPorPeriodo', axis=1, inplace=True)
# df_fato.sample(5).T

## CRA por Periodo

In [107]:
df['craPorPeriodo'] = df['craPorPeriodo'].astype(str)
df['craPorPeriodo'] = df['craPorPeriodo'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])
df_exploded = df.explode('craPorPeriodo')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...,2000/1 - MAA123 Algebra para Informatica - 060...
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"(2000/1, 7.6)","(2000/2, 7.4)","(2001/1, 6.9)","(2001/2, 6.9)","(2002/1, 6.8)"


In [108]:
df_craPeriodo = pd.DataFrame(df_exploded['craPorPeriodo'].tolist(), index=df_exploded.index, columns=('periodo','cra'))
df_craPeriodo['matriculaDRE'] = df_exploded['matriculaDRE']
df_craPeriodo

,periodo,cra,matriculaDRE
0,2000/1,7.6,0
0,2000/2,7.4,0
0,2001/1,6.9,0
0,2001/2,6.9,0
0,2002/1,6.8,0
...,...,...,...
2775,2022/1,7.5,2775
2776,2022/1,7.8,2776
2777,2022/1,7.9,2777
2778,2022/1,6.7,2778


In [109]:
# df_fato.drop('craPorPeriodo', axis=1, inplace=True)
# df_fato = pd.merge(df_fato, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
# df_fato.head(5).T

## Disciplinas Cursadas

Nessa seção vamos separar a lista de disciplinas em varias colunas, uma para cada disciplina

In [110]:
df['disciplinasCursadas'] = df['disciplinasCursadas'].astype(str)
df['disciplinasCursadas'] = df['disciplinasCursadas'].apply(lambda x: [tuple(item.split(' - ')) for item in x.split('\n')])

In [111]:
df_exploded = df.explode('disciplinasCursadas')
df_exploded.head(5).T

,0,0,0,0,0
matriculaDRE,0,0,0,0,0
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
periodoIngressoCursoAtual,2000/1,2000/1,2000/1,2000/1,2000/1
situacaoMatriculaAtual,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso,Cancelada por conclusão de curso
sexo,M,M,M,M,M
disciplinasCursadas,"(2000/1, MAA123 Algebra para Informatica, 060,...","(2000/1, MAB111 Fund da Computação Digital, 08...","(2000/1, MAB120 Computacao para Informatica, 0...","(2000/1, MAE111 Cálculo Infinitesimal I, 056, ...","(2000/1, MAE115 Cálculo Vetorial e G Analitica..."
crPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ...","[(2000/1, 7.6), (2000/2, 7.1), (2001/1, 5.9), ..."
craPorPeriodo,"[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ...","[(2000/1, 7.6), (2000/2, 7.4), (2001/1, 6.9), ..."


Agora vamos pegar as informações de cada desciplina e desmembrar em novas colunas

In [112]:
df_exploded['disciplinasCursadas'] = df_exploded['disciplinasCursadas'].apply(lambda x: (
    x[0].replace(' ', ''),
    x[1].split()[0].replace(' ', ''),
    ' '.join(x[1].split()[1:]),
    x[2].replace(' ', ''),
    x[3].replace(' ', '')
))

In [113]:
df_disciplina = pd.DataFrame(df_exploded['disciplinasCursadas'].tolist(), index=df_exploded.index, columns=('periodo','cod_Disciplina','disciplina','grau_disciplina','situacao_disciplina'))
df_disciplina['matriculaDRE'] = df_exploded['matriculaDRE']
df_disciplina

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0
0,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0
0,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0
0,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0
0,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0
...,...,...,...,...,...,...
2779,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779
2779,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779
2779,2022/1,ICP120,Computação I (CC),075,Aprovado,2779
2779,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779


In [114]:
# df_fato.drop('disciplinasCursadas',axis=1, inplace=True)
# df_fato = pd.merge(df_fato, df_disciplina, on=['periodo', 'matriculaDRE'], how='left')
# df_fato.head(5).T

In [115]:
df_merge = pd.merge(df_disciplina, df_craPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge = pd.merge(df_merge, df_crPeriodo, on=['periodo', 'matriculaDRE'], how='left')
df_merge

,periodo,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cra,cr
0,2000/1,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6
1,2000/1,MAB111,Fund da Computação Digital,089,Aprovado,0,7.6,7.6
2,2000/1,MAB120,Computacao para Informatica,094,Aprovado,0,7.6,7.6
3,2000/1,MAE111,Cálculo Infinitesimal I,056,Aprovado,0,7.6,7.6
4,2000/1,MAE115,Cálculo Vetorial e G Analitica,085,Aprovado,0,7.6,7.6
...,...,...,...,...,...,...,...,...
98634,2022/1,ICP111,Fund Computação Digital,067,Aprovado,2779,7.8,7.8
98635,2022/1,ICP112,Sistemas de Informação,082,Aprovado,2779,7.8,7.8
98636,2022/1,ICP120,Computação I (CC),075,Aprovado,2779,7.8,7.8
98637,2022/1,ICP136,Introd Pensamento Dedutivo,081,Aprovado,2779,7.8,7.8


In [116]:
df_fato = pd.merge(df_merge, df, on=['matriculaDRE'], how='left')
df_fato.drop(['disciplinasCursadas','crPorPeriodo', 'craPorPeriodo'], axis=1, inplace=True)
df_fato.sample(5).T

,96307,93869,8399,31508,95124
periodo,2021/1,2020/1,2006/1,2009/2,2022/1
cod_Disciplina,CMT231,MAB116,MAB601,FIM231,ICP122
disciplina,Inovações Cient Tecn e Art III,Estrutura dos Dados,Top Esp em Eng de Software,Física III,Computação Gráfica I
grau_disciplina,085,092,100,015,086
situacao_disciplina,Aprovado,Aprovado,Aprovado,Reprovadomedia,Aprovado
matriculaDRE,2543,2424,197,722,2479
cra,8.1,7.9,7.9,4.7,7.3
cr,8.5,8.6,8.8,0.5,6.1
periodoIngressoUFRJ,2020/1,2019/1,2001/1,2005/1,2019/2
cursoIngressoUFRJ,Bach em Ciências Matemáticas e da Terra,Bacharelado em Ciência da Computação,Bacharelado em Informática,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [122]:
df_fato[['ano', 'semestre']] = df_fato['periodo'].str.split('/', expand=True)
df_fato.head(5).T

,0,1,2,3,4
periodo,2000/1,2000/1,2000/1,2000/1,2000/1
cod_Disciplina,MAA123,MAB111,MAB120,MAE111,MAE115
disciplina,Algebra para Informatica,Fund da Computação Digital,Computacao para Informatica,Cálculo Infinitesimal I,Cálculo Vetorial e G Analitica
grau_disciplina,060,089,094,056,085
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
matriculaDRE,0,0,0,0,0
cra,7.6,7.6,7.6,7.6,7.6
cr,7.6,7.6,7.6,7.6,7.6
periodoIngressoUFRJ,2000/1,2000/1,2000/1,2000/1,2000/1
cursoIngressoUFRJ,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática,Bacharelado em Informática


# Dimenssão Aluno

In [123]:
colunas_aluno = ['sk_d_aluno', 'nome_completo', 'matriculaDRE', 'sexo']

In [124]:
d_aluno = df_fato[colunas_aluno[1:]]

# d_aluno.groupby(['nome_completo', 'matriculaDRE', 'sexo']).first().reset_index()
d_aluno = d_aluno.drop_duplicates().reset_index(drop=True)
d_aluno

,nome_completo,matriculaDRE,sexo
0,Kaique Freitas,0,M
1,Luiz Fernando Barros,1,M
2,Ana Júlia Novaes,2,M
3,Dra. Carolina Fernandes,3,M
4,Ana Luiza da Mota,4,M
...,...,...,...
2645,João Miguel Santos,2775,M
2646,Caroline Moura,2776,M
2647,João Monteiro,2777,M
2648,Gabriel Ribeiro,2778,M


In [125]:
d_aluno['sk_d_aluno'] = d_aluno.apply(lambda x: hashlib.sha256(str(x['matriculaDRE']).encode()).hexdigest(), axis=1)
d_aluno = d_aluno.reindex(columns=colunas_aluno)
d_aluno.to_csv('data/d_aluno.csv', index=False)
d_aluno

,sk_d_aluno,nome_completo,matriculaDRE,sexo
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,Kaique Freitas,0,M
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,Luiz Fernando Barros,1,M
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,Ana Júlia Novaes,2,M
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,Dra. Carolina Fernandes,3,M
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,Ana Luiza da Mota,4,M
...,...,...,...,...
2645,3da16474e33b330b71e5b562cfcf7939bde3c12a08b746...,João Miguel Santos,2775,M
2646,9b3cea3bd8789ba47cc969bff5914da7ebac8e0052e282...,Caroline Moura,2776,M
2647,5a48eed290f62c93553855c36c964e1ef16603d23dcce3...,João Monteiro,2777,M
2648,a2541e84e912f4f5dd32ce131abdb6b1fdc9cf315790f1...,Gabriel Ribeiro,2778,M


# Dimenssão Disciplina

In [537]:
# def criar_dimenssao(nome_dimenssao, colunas_dimenssao, hash):
#     dimenssao = df_fato[colunas_dimenssao[1:]]
#     dimenssao = dimenssao.drop_duplicates().reset_index(drop=True)
#     dimenssao[f'sk_d_{nome_dimenssao}'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x[hash]).encode()).hexdigest(), axis=1)
#     d_disciplina = d_disciplina.reindex(columns=colunas_aluno)
#     d_disciplina.to_csv(f'data/d_{nome_dimenssao}.csv', index=False)
#     d_disciplina

In [126]:
colunas_disciplina = ['sk_d_disciplina', 'cod_Disciplina', 'disciplina']

In [244]:
d_disciplina = df_fato[colunas_disciplina[1:]]
d_disciplina = d_disciplina.drop_duplicates().reset_index(drop=True)
d_disciplina

,cod_Disciplina,disciplina
0,MAA123,Algebra para Informatica
1,MAB111,Fund da Computação Digital
2,MAB120,Computacao para Informatica
3,MAE111,Cálculo Infinitesimal I
4,MAE115,Cálculo Vetorial e G Analitica
...,...,...
1589,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,MAE245,Álgebra Linear II
1592,ICP114,Computação I Ep


In [245]:
d_disciplina['sk_d_disciplina'] = d_disciplina.apply(lambda x: hashlib.sha256(str(x['cod_Disciplina'] + x['disciplina']).encode()).hexdigest(), axis=1)
d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)
d_disciplina.to_csv('data/d_disciplina.csv', index=False)
d_disciplina

,sk_d_disciplina,cod_Disciplina,disciplina
0,26ae25e81ccc85aecdad201a410a09485d415acd4daf6e...,MAA123,Algebra para Informatica
1,4ba51a95ba6087306319254f45d5ee5ad335922069282f...,MAB111,Fund da Computação Digital
2,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,MAB120,Computacao para Informatica
3,63637e47a78400c87bb1b82da2fc5b6c00e78c0c77c779...,MAE111,Cálculo Infinitesimal I
4,6f904cf605935af92e93713c9eead620c9c92cd971dfc4...,MAE115,Cálculo Vetorial e G Analitica
...,...,...,...
1589,f711fb63fc524f5758c111314b49b44d86a21e48bb458e...,NEP144,Tóp Esp Polít Públ Dir Hum VI
1590,eac458f82acdd92fd7aa393c4453804d5516adbbaa6158...,NEP142,Tóp Esp Polít Públ Dir Hum IV
1591,5bda2447fb592bd1f9f7cbd461bc602ff7292bacf42222...,MAE245,Álgebra Linear II
1592,9d40b24dedf530a6991c38a516e1275f152e62264a656e...,ICP114,Computação I Ep


# Dimenssão Periodo

In [129]:
colunas_periodo = ['sk_d_periodo', 'ano', 'semestre']

In [138]:
d_periodo = df_fato[colunas_periodo[1:]]
d_periodo = d_periodo.drop_duplicates().reset_index(drop=True)
d_periodo

,ano,semestre
0,2000,1
1,2000,2
2,2001,1
3,2001,2
4,2002,1
...,...,...
67,2019,0
68,2021,0
69,2013,3
70,2014,3


In [139]:
d_periodo['sk_d_periodo'] = d_periodo.apply(lambda x: hashlib.sha256(str(x['ano'] + x['semestre']).encode()).hexdigest(), axis=1)
d_periodo = d_periodo.reindex(columns=colunas_periodo)
d_periodo.to_csv('data/d_periodo.csv', index=False)
d_periodo

,sk_d_periodo,ano,semestre
0,fc82267b45dcbf8db9e4ec10055373cfff41efcbc5b83a...,2000,1
1,29433eae6f7f1308d9799275f3a90a0afe1fef0e1818a7...,2000,2
2,e4b8f2da99fd9407c758cd306f846562424ec5c38d3202...,2001,1
3,b6167a59dbfdca3d716a326a015dd8a00f417590eb34e2...,2001,2
4,c291fe829d40dc63fc4fa578f6208f425064945d31ca82...,2002,1
...,...,...,...
67,8ad00ecd5eb3a6ca42f90cc776b009968ec5865f2d37b3...,2019,0
68,fa122eae077795ac4ff0f80bb75550a9a2c30a192d035c...,2021,0
69,fa103adf3e582291297b8330060eace6bc1a14f8737cf2...,2013,3
70,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,2014,3


# Dimenssão Curso

In [215]:
colunas_curso = ['sk_d_curso', 'cursoIngressoUFRJ', 'cursoAtual']

In [216]:
d_curso = df_fato[colunas_curso[1:]]
d_curso = d_curso.drop_duplicates().reset_index(drop=True)
d_curso

,cursoIngressoUFRJ,cursoAtual
0,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,Bacharelado em Informática,Bacharelado em Informática
2,Bacharelado em Informática,Licenciatura em Matemática
3,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,Matemática,Bacharelado em Ciência da Computação
...,...,...
73,Licenciatura em Física,Bacharelado em Ciência da Computação
74,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,Ciências Matemáticas e da Terra,Ciência da Computação
76,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


In [217]:
d_curso['sk_d_curso'] = d_curso.apply(lambda x: hashlib.sha256(str(x['cursoIngressoUFRJ'] + x['cursoAtual']).encode()).hexdigest(), axis=1)
d_curso = d_curso.reindex(columns=colunas_curso)
d_curso.to_csv('data/d_curso.csv', index=False)
d_curso

,sk_d_curso,cursoIngressoUFRJ,cursoAtual
0,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,Bacharelado em Informática,Bacharelado em Ciência da Computação
1,dbde55e27224a8c832119eba0c54cb25a32bf6f6417dd2...,Bacharelado em Informática,Bacharelado em Informática
2,cb369a9ae66bcd4659f91ed5d88c2a7674ca54645d641a...,Bacharelado em Informática,Licenciatura em Matemática
3,0367c21aa27e715610d936288c8fb811fc6dd378b050b5...,Bacharelado em Informática,Mat Aplicada - Computação Científica
4,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,Matemática,Bacharelado em Ciência da Computação
...,...,...,...
73,b04e50e1e81b07e4bba26b99505e3a21fd9160bdd6c664...,Licenciatura em Física,Bacharelado em Ciência da Computação
74,2d37b7f8c6308826f3cd805e314628baf4dede1e70bb0a...,Bacharelado em Ciência da Computação,Licenciatura em Geografia
75,acbc30f06ec39601955703a6f4fb05791a031a8f7755d1...,Ciências Matemáticas e da Terra,Ciência da Computação
76,a54d2fed0213e2b06b6fb51bf134c8f5920420505116c3...,Bacharelado em Ciência da Computação,Engenharia Eletrônica e de Computação


# Dimenssão Situação Academica

In [152]:
colunas_situacao_academica = ['sk_d_situacao_academica', 'situacaoMatriculaAtual']

In [153]:
d_situacao_academica = df_fato[colunas_situacao_academica[1:]]
d_situacao_academica = d_situacao_academica.drop_duplicates().reset_index(drop=True)
d_situacao_academica

,situacaoMatriculaAtual
0,Cancelada por conclusão de curso
1,Cancelada por abandono
2,Cancelada por transferência
3,Cancelada por abandono Definitivo
4,Cancelamento a pedido
5,Cancelamento por ultrapassagem do prazo máximo...
6,Cancelamento por opção de curso
7,Cancelada por rendimento escolar insuficiente
8,Cancelamento por conclusão de Mobilidade Acadê...
9,Rematrícula por destrancamento ou descancelamento


In [155]:
d_situacao_academica['sk_d_situacao_academica'] = d_situacao_academica.apply(lambda x: hashlib.sha256(str(x['situacaoMatriculaAtual']).encode()).hexdigest(), axis=1)
d_situacao_academica = d_situacao_academica.reindex(columns=colunas_situacao_academica)
d_situacao_academica.to_csv('data/d_situacao_academica.csv', index=False)
d_situacao_academica

,sk_d_situacao_academica,situacaoMatriculaAtual
0,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,Cancelada por conclusão de curso
1,3ca80f8447d1816b42e6b6ce78876a09ea7ad5e0ae301f...,Cancelada por abandono
2,8362ba253c1a3896ff7da3aaab2e175cb28646495c5e63...,Cancelada por transferência
3,75aeed555b3a43b6e0dddc2c40fd7475c412508eeb27d2...,Cancelada por abandono Definitivo
4,27a82bbf3475a3ffcd91c78ad15adc39fff1f210596c27...,Cancelamento a pedido
5,8336247db87007dd0d6673c56d524eac7df89beccaa6a6...,Cancelamento por ultrapassagem do prazo máximo...
6,924e940f4a6c290b22e8b4a9c1d554a38cfd80b8364c02...,Cancelamento por opção de curso
7,2919d1d61aa9bcc127530e38fc132e937b8b43d21a1938...,Cancelada por rendimento escolar insuficiente
8,5e723f9d214270c85a578cee66bfb97b78dea411c14fa3...,Cancelamento por conclusão de Mobilidade Acadê...
9,e6ec8667d6f4052cd7f8b0691be380ed691e49c3351995...,Rematrícula por destrancamento ou descancelamento


# Fato Desempenho Academico

In [254]:
f_desempenho_academico = df_fato.drop(['periodo', 'periodoIngressoCursoAtual', 'periodosIntegralizados', 'cargaHorariaAcumulada'], axis=1)
f_desempenho_academico.head(1)

,cod_Disciplina,disciplina,grau_disciplina,situacao_disciplina,matriculaDRE,cra,cr,periodoIngressoUFRJ,cursoIngressoUFRJ,cursoAtual,situacaoMatriculaAtual,sexo,nome_completo,ano,semestre
0,MAA123,Algebra para Informatica,060,Aprovado,0,7.6,7.6,2000/1,Bacharelado em Informática,Bacharelado em Ciência da Computação,Cancelada por conclusão de curso,M,Kaique Freitas,2000,1


In [255]:
# D_SITUACAO_ACADEMICA
f_desempenho_academico = pd.merge(f_desempenho_academico, d_situacao_academica, on=['situacaoMatriculaAtual'], how='left')
f_desempenho_academico.drop(['situacaoMatriculaAtual'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,32216,50423,33199,38177,37590
cod_Disciplina,MAB637,MAE993,MAB120,MAB230,FIT111
disciplina,Governança da Internet,Cálculo Int e Diferencial III,Computacao para Informatica,Cálculo Numérico (CC),Física I
grau_disciplina,080,015,086,083,000
situacao_disciplina,Aprovado,Reprovadomedia,Aprovado,Aprovado,Reprfalta/media
matriculaDRE,737,1160,763,873,860
cra,6.6,3.6,7.2,5.4,2.2
cr,6.9,0.6,7.2,7.3,0.3
periodoIngressoUFRJ,2005/2,2009/1,2005/2,2006/2,2006/2
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
cursoAtual,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [256]:
# D_PERIODO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_periodo, on=['ano', 'semestre'], how='left')
f_desempenho_academico.drop(['ano', 'semestre'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,86457,65046,39423,38708,50130
cod_Disciplina,MAB113,IQG231,MAB124,MAB519,MAB608
disciplina,Organização da Informação,Química Experimental EQ,Programação de Computadores I,Informática Aplicada ao Ensino,Top Esp Intelig Computacional
grau_disciplina,092,000,100,084,080
situacao_disciplina,Aprovado,Reprovadomedia,Aprovado,Aprovado,Aprovado
matriculaDRE,2160,1577,905,886,1155
cra,6.5,2.7,5.6,7.1,6.8
cr,7.8,1.2,5.8,7.7,6.9
periodoIngressoUFRJ,2017/1,2012/1,2007/1,2006/2,2009/1
cursoIngressoUFRJ,Bacharelado em Ciência da Computação,Engenharia Química,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação
cursoAtual,Relações Internacionais,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação,Bacharelado em Ciência da Computação


In [257]:
# D_CURSO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_curso, on=['cursoIngressoUFRJ', 'cursoAtual'], how='left')
f_desempenho_academico.drop(['cursoIngressoUFRJ', 'cursoAtual'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,40129,51384,46178,57887,3738
cod_Disciplina,MAB366,MAB525,MAB122,MAB515,MAB368
disciplina,Sistemas Operacionais I,Tópicos Esp em Int Artificial,Computação Gráfica I,Avaliação e Desempenho,Algoritmos e Grafos
grau_disciplina,083,063,069,010,067
situacao_disciplina,Aprovado,Aprovado,Aprovado,Reprfalta/media,Aprovado
matriculaDRE,920,1189,1056,1405,79
cra,8.9,7.3,8.3,5.7,7
cr,8,7.1,8.5,1,7.4
periodoIngressoUFRJ,2007/1,2009/1,2008/1,2010/2,2000/2
sexo,M,M,M,M,M
nome_completo,Luiz Otávio Ferreira,Yago Silveira,Heloísa Moraes,Caio Nascimento,André Cardoso


In [258]:
# D_ALUNO
f_desempenho_academico = pd.merge(f_desempenho_academico, d_aluno, on=['nome_completo', 'sexo', 'matriculaDRE'], how='left')
f_desempenho_academico.drop(['nome_completo', 'sexo', 'matriculaDRE'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,83215,11527,86239,19769,43061
cod_Disciplina,FAH473,MABX04,MAB113,MAB245,FIT111
disciplina,Arquitetura no Brasil II,Monitoria II,Organização da Informação,Circuitos Lógicos,Física I
grau_disciplina,085,100,090,066,051
situacao_disciplina,Aprovado,Aprovado,Aprovado,Aprovado,Aprovado
cra,6.8,7.7,7.5,6.8,5
cr,8.8,7.9,7.3,7.1,4.6
periodoIngressoUFRJ,2016/1,2002/1,2017/1,2003/1,2007/2
sk_d_situacao_academica,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,e6ec8667d6f4052cd7f8b0691be380ed691e49c3351995...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,8362ba253c1a3896ff7da3aaab2e175cb28646495c5e63...
sk_d_periodo,1d1608805160ee7f833c28de50dd6887e8910ae2a7418f...,7ed6967ca706ff12a3f16ffe89128eac5f23db35b6e22e...,dddd8a22f99ec19d35bd2f6c9714ba305614d47f3ca371...,551f1b6da3d076f00cf0f65b0bbb08ee6e88d26efd4abf...,9652f0351390fbe354fe9370bc85f99e126751f682ccfd...
sk_d_curso,a2bf045b5cd2a392636964653bf6c73fda2e755117cd6a...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,a80f5dd573274667573b525e0750e7915a3bc967190075...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,69f98f9a931103c9534d1881425cd98f8d920686439325...


In [259]:
# D_DISCIPLINA
f_desempenho_academico = pd.merge(f_desempenho_academico, d_disciplina, on=['cod_Disciplina', 'disciplina'], how='left')
f_desempenho_academico.drop(['cod_Disciplina', 'disciplina'], axis=1, inplace=True)
f_desempenho_academico.sample(5).T

,73290,15516,63590,53278,49417
grau_disciplina,100,050,035,084,045
situacao_disciplina,Aprovado,Aprovado,Reprovadomedia,Aprovado,Reprovadomedia
cra,NaN,5.3,4.5,4.2,5.1
cr,NaN,5.3,5.4,6,5.3
periodoIngressoUFRJ,2013/2,2002/2,2011/2,2009/2,2008/2
sk_d_situacao_academica,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,8336247db87007dd0d6673c56d524eac7df89beccaa6a6...,65cb84f03d1e58898e1ea2b11194c5ff6096ad76b600de...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...
sk_d_periodo,10071c57cf3b2135572815c0243a079fc18936a1259c3d...,c2db217ea7a34191c8f55a2acd4ac37c35611dc1946a7c...,dffcc977a35e3e941a5567f71d111418e49a25109152d0...,12ae265fcfe315a363b94cb06b5634257af6517781d59a...,b18c61c55f17af492ac0535bf17d89dfb521c9110d0cbd...
sk_d_curso,7136f5be93722f0a5841806277fcfa918ced92179b21bf...,74ea6a4ffce0cb8d3013a9978cfd9db27b906f4006c1f2...,b73a0b12bf115c48af094f97260abc5aeac85cc2e47360...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...
sk_d_aluno,d6245609f9341aa8022554b45ea8d79e90ea83184e739b...,600b4cdf20cc06a7b5a5cca5f7464296861815519af6d8...,596d0c702ba6d20835982c87a878f97d07a163285a0a3e...,665cb02ac3b3b46848dff3760744b8c87710eef31b8e73...,96be21608a6aa9d2424e4af66984baf4a32690789ed4f3...
sk_d_disciplina,535637d15185e8b51a14096d3907252748b5cb412f6c29...,09e5060ee94ea2b9604e23295927e68d40d481a362d81e...,de4f94b5804ebaccfa8f5d68972cfbd7693a996d28f224...,7a93cfa230c45539ca98578d5e86ebda2cd687764ee847...,def4682d25ac08dff9e665120fd38abc76b1eef99aece1...


In [260]:
f_desempenho_academico.drop(['periodoIngressoUFRJ'], axis=1, inplace=True)

In [261]:
f_desempenho_academico.sample(5).T

,24462,30986,55306,31208,87921
grau_disciplina,096,060,015,075,074
situacao_disciplina,Aprovado,Aprovado,Reprfalta/media,Aprovado,Aprovado
cra,7.9,5,5.2,8.7,5.6
cr,7.5,3.3,3.5,8.5,7.3
sk_d_situacao_academica,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,d01fc610cec0ea644fb1e3ee88a57515590e0c47daa640...
sk_d_periodo,8d5baa326957041e61222d78ab30f209846b0a6c591c1b...,0eb55bc6d6f2d19b431c9c319b0f595e3d2cc988d0f9ae...,e54a2041a494a6534e33f777b64c7912f8a2cc5a4e9a33...,0eb55bc6d6f2d19b431c9c319b0f595e3d2cc988d0f9ae...,70d6309c8a5c913ad3ef3800beae7ccac93955bd451810...
sk_d_curso,69f98f9a931103c9534d1881425cd98f8d920686439325...,5f20305be16f0143bc9a1bcc213bc104dd93a64e5ad031...,69f98f9a931103c9534d1881425cd98f8d920686439325...,69f98f9a931103c9534d1881425cd98f8d920686439325...,a80f5dd573274667573b525e0750e7915a3bc967190075...
sk_d_aluno,4eef24c6b8248c2271f6663f44ec0de3c2535ca396a22c...,2499d690642faa4da2a67b078236d1c031217f3c31cf2d...,7ba7d31bfa1ed86327ecfa9deb2dd8a44488fba943ca78...,35c71bd7eaf4607047bb7c186d17251942204229b897e0...,85987aa6506068cbd41d277440422c66c0e32b77591035...
sk_d_disciplina,2461c79130f623f026b98adb94ada036e9d8c18fc4f5b5...,2461c79130f623f026b98adb94ada036e9d8c18fc4f5b5...,2f49ad3415ebb9ec607179953d5d7d923477a6af18b3fc...,24fe68dd71397b411b564b2faf1db31786d3296fdea5e9...,2993d0e1bb6ecd5e29879451e7463a05cc6075da0e70d7...


In [ ]:
f_desempenho_academico.to_csv('data/f_desempenho_academico.csv', index=False)

## Analise de disciplinas com nomes errados

In [234]:
d_disciplina[d_disciplina['cod_Disciplina'] == 'MAB613']

,sk_d_disciplina,cod_Disciplina,disciplina
51,fa2b85c8f7ce6cd8e6258d773e7f63312659803e5a903c...,MAB613,Met Desenv de Sist Informacao
118,fa2b85c8f7ce6cd8e6258d773e7f63312659803e5a903c...,MAB613,Met Desenv de Sist Informação


In [243]:
d_disciplina[d_disciplina['cod_Disciplina'] == 'MAB601']

,sk_d_disciplina,cod_Disciplina,disciplina
50,3a74ed7d699c56d55413c7e303a77238e9919a287b8353...,MAB601,Top Esp em Eng de Softaware
61,3a74ed7d699c56d55413c7e303a77238e9919a287b8353...,MAB601,Top Esp em Eng de Software


In [230]:
d_disciplina[d_disciplina['cod_Disciplina'].duplicated()]

,sk_d_disciplina,cod_Disciplina,disciplina
61,3a74ed7d699c56d55413c7e303a77238e9919a287b8353...,MAB601,Top Esp em Eng de Software
118,fa2b85c8f7ce6cd8e6258d773e7f63312659803e5a903c...,MAB613,Met Desenv de Sist Informação
358,36798eb1dd4bca5690dddb47a9e76ce26a7e11c49f1f7a...,MAB603,Gestão do Conhecimento
383,e1feda02d4435e105b01d48a0d5ccba6126d822c25a6a2...,MAB230,Cálculo Numérico (CC)
417,f665961e92aa45d6246dc7afaf2bfc6ea6e1b81ed4b3d1...,MAB602,Dw no Sup à Tomada de Decisão
419,64fc075bdc55d380e90a786c31e9d698e048ce4be5f710...,MAB600,Dados Semi-estruturados e XML
460,f6cb44b56a71039dc9d15312ea8a1d9fd28c3f179f3958...,MAB532,Mineração de Dados
462,0995b6821e0db11cf40caec82648bf487ba4e9fae1386e...,EEH210,Engenharia e Meio Ambiente
471,135390137e887141f44e1863d0819b245131ee4eb097e4...,IEE304,Desenvolvimento Socioeconomico
474,8c0e7f3209f34c0e3dbd9901137fda665888f0abb015bd...,MAB120,Computação I (CC)
